In [1]:
import os
os.environ['PIP_CACHE_DIR'] = '/scratch/ola5/cache'
os.environ['HF_HOME'] = '/scratch/ola5/cache'

In [2]:
%cd Open-Sora-Plan

/scratch/ola5/opensora/Open-Sora-Plan


/scratch/ola5/opensora/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
weight_dtype = torch.bfloat16

from diffusers.schedulers import DPMSolverMultistepScheduler, EulerAncestralDiscreteScheduler
from transformers import T5EncoderModel, MT5EncoderModel, AutoTokenizer

from opensora.models.causalvideovae import ae_stride_config, ae_channel_config, ae_norm, ae_denorm, CausalVAEModelWrapper
from opensora.models.diffusion.opensora.modeling_opensora import OpenSoraT2V
#from opensora.sample.pipeline_opensora import OpenSoraPipeline
from opensora.sample.pipeline2 import OpenSoraPipeline

import imageio

/scratch/ola5/opensora/lib/python3.10/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/scratch/ola5/opensora/lib/python3.10/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


[2024-08-13 01:13:22,742] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.1
 [WARNING]  using untested triton version (2.1.0), only 1.0.0 is known to be compatible
The npu_config.on_npu is False
pid 25840's current affinity list: 0-31
pid 25840's new affinity list: 0-3


/scratch/ola5/opensora/lib/python3.10/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
/scratch/ola5/opensora/lib/python3.10/site-packages/diffusers/models/transformer_2d.py:20: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 0.29. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.transformers.transformer_2d import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "0.29", deprecation_message)
/scratch/ola5/opensora/lib/python3.10/site-packages/diffusers/models/transformer_2d.py:25: FutureWarning: `Transformer2DModel` is

In [4]:
transformer_model = OpenSoraT2V.from_pretrained("./models/93x720p", cache_dir='./cache_dir', low_cpu_mem_usage=False, device_map=None, torch_dtype=weight_dtype)

The config attributes {'decay': 0.999, 'inv_gamma': 1.0, 'min_decay': 0.0, 'optimization_step': 400, 'power': 0.6666666666666666, 'update_after_step': 0, 'use_ema_warmup': False} were passed to OpenSoraT2V, but are not expected and will be ignored. Please verify your config.json configuration file.


In [5]:
transformer_model.eval()
scheduler = EulerAncestralDiscreteScheduler()

pipeline = OpenSoraPipeline(scheduler=scheduler, transformer=transformer_model)
pipeline.to(device)

OpenSoraPipeline {
  "_class_name": "OpenSoraPipeline",
  "_diffusers_version": "0.28.0",
  "scheduler": [
    "diffusers",
    "EulerAncestralDiscreteScheduler"
  ],
  "transformer": [
    "opensora.models.diffusion.opensora.modeling_opensora",
    "OpenSoraT2V"
  ]
}

In [ ]:
#positive_prompt = """
#(masterpiece), (best quality), (ultra-detailed),
#{}.
#"""
#negative_prompt = """
#lowres, bad anatomy, cropped, worst quality, low quality, jpeg artifacts, signature, watermark, username, blurry
#"""

import pickle
with open('prompt.pkl', 'rb') as f:
    data = pickle.load(f)

num_frames = 93
latents = pipeline(
    negative_prompt=None,
    num_frames=num_frames,
    height=720,
    width=1280,
    num_inference_steps=75,
    guidance_scale=7.5,
    num_images_per_prompt=1,
    device=device,
    prompt_embeds=data[0],
    prompt_attention_mask=data[1],
    negative_prompt_embeds=data[2],
    negative_prompt_attention_mask=data[3],
    max_sequence_length=300,
)

  0%|          | 0/75 [00:00<?, ?it/s]

In [7]:
latents.shape

torch.Size([1, 4, 24, 90, 160])

In [8]:
with open('latents.pkl', 'wb') as f:
    pickle.dump(latents, f)

In [9]:
latents.dtype, latents.device

(torch.bfloat16, device(type='cuda', index=0))